In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


###Load the data.

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


###Look at the median_house_value variable. Does it have a long tail?
Yes, it has a long tail.

In [4]:
df['median_house_value']

0        452600.0
1        358500.0
2        352100.0
3        341300.0
4        342200.0
           ...   
20635     78100.0
20636     77100.0
20637     92300.0
20638     84700.0
20639     89400.0
Name: median_house_value, Length: 20640, dtype: float64

###Question 1
There's one feature with missing values. What is it?

- total_rooms
- *total_bedrooms*
- population
- households

answer: **total_bedrooms**

In [6]:
na=df.isna()
values=na.sum()
print(values) 

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64


Question 2
What's the median (50% percentile) for variable 'population'?

- 995
- 1095
- 1195
- 1295

answer: 

In [9]:

"""filtered_data = df[df['population'] > 1000]

mixed_data = filtered_data.sample(frac=1, random_state=42).reset_index(drop=True)


train_size = int(0.6 * len(mixed_data))
val_size = int(0.2 * len(mixed_data))


train_data = mixed_data[:train_size]
val_data = mixed_data[train_size:train_size + val_size]
test_data = mixed_data[train_size + val_size:]

median_population = filtered_data['population'].median()
print(median_population)"""

1573.0
